In [1]:
import time
from datetime import datetime
import pandas as pd

notebook_start = time.time()

In [2]:
dataframes =pd.ExcelFile("Hari Mobile chatbot.xlsx")
question = pd.read_excel(dataframes, 'question ')

In [3]:
question_df = question[['question ','part']]
question_df.shape

(299, 2)

In [4]:
question_df.head()
question_df = question_df.dropna()
question_df.shape

(214, 2)

In [5]:
services = ['combo', 'service', 'display', 'motherboard','flash box','file']
for index,rows in question_df.iterrows():
    if rows['part'] not in services:
        question_df.at[index,'part']="others"
    if rows['part']=='display':
        question_df.at[index,'part']="combo"

In [6]:
question_df['part'].value_counts()

service        100
combo           32
others          30
flash box       25
file            19
motherboard      8
Name: part, dtype: int64

In [7]:
# pd.options.display.max_colwidth = 1000
# question_df.loc[question_df['part']=='combo']

In [8]:
question_df['part'].value_counts()

service        100
combo           32
others          30
flash box       25
file            19
motherboard      8
Name: part, dtype: int64

In [9]:
question_df['part'].value_counts(normalize=True)

service        0.467290
combo          0.149533
others         0.140187
flash box      0.116822
file           0.088785
motherboard    0.037383
Name: part, dtype: float64

In [10]:
# parts = ['service','combo','flash box','file','motherboard','country unlock sim','ic',
#          'display','battery','mobile','flash tool']
# parts = ['service','combo','flash box','file','motherboard','others']
# question_df = question_df.loc[question_df['part'].isin(parts)]

In [11]:
question_df.shape

(214, 2)

In [12]:
from nodes import feature_generation, metrics, naive_bayes

Using TensorFlow backend.
/home/viky/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/viky/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/viky/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/viky/.local/lib/python3.7/site-packages/tensorflow/python/framewo

In [13]:
parameters= {'random_seed': 121,
 'merge_labels': {'self_service': ['claim_self_service',
   'policy_self_service'],
  'claim_enquiry': ['claim_payment', 'claim_update']},
 'test_data_ratio': 0.2,
 'subsample_factor_ova': 2,
 'feature_generation_naivebayes': {'remove_stopwords': False,
  'tfidf': True,
  'ngram_start': 1,
  'ngram_end': 3,
  'max_features': 800},
 'train_naivebayes': {'name': 'naivebayes_unigram_model'}}

In [14]:
collated_data = question_df

In [15]:
from importlib import reload
train_data, test_data = feature_generation.split_data(collated_data, parameters)

print( train_data.shape, test_data.shape)

(171, 2) (43, 2)


In [16]:
test_data['part'].value_counts()

service        18
combo           9
others          8
flash box       4
motherboard     3
file            1
Name: part, dtype: int64

In [17]:
from importlib import reload
reload(feature_generation)

train_x, train_y, test_x, test_y, known_categories, vectorizer, train_pred, test_pred = \
feature_generation.generate_ngram_features(train_data, test_data, parameters['feature_generation_naivebayes'])

In [18]:
known_categories

['service', 'others', 'combo', 'file', 'motherboard', 'flash box']

In [19]:
from importlib import reload
reload(naive_bayes)
reload(feature_generation)

model = naive_bayes.train(train_x, train_y, test_x, test_y, parameters["train_naivebayes"])

pred_label_test, pred_label_test_dec, pred_probs, test_pred = naive_bayes.predictions(test_x, test_pred, model, known_categories)

pred_label_train, pred_label_train_dec, pred_probs, train_pred = naive_bayes.predictions(train_x, train_pred, model, known_categories)

(43, 800)
(171, 800)


In [20]:
train_x

<171x800 sparse matrix of type '<class 'numpy.float64'>'
	with 2129 stored elements in Compressed Sparse Row format>

In [21]:
test_pred.loc[(test_pred.part=='service')& (test_pred.pred_label=='combo')]

,question,part,label_enc,pred_label,pred_label_enc,pred_label_prob
179,💯% og care og batteries Available only this mo...,service,0,combo,2,0.406281
226,A3s ho jayega kya,service,0,combo,2,0.386275


In [22]:
from importlib import reload
reload(metrics)

metrics_train = metrics.get_all_error_metrics(train_y, pred_label_train, known_categories)

metrics_test = metrics.get_all_error_metrics(test_y, pred_label_test, known_categories)

accuracy 0.9473684210526315
accuracy 0.627906976744186


In [23]:
train_x

<171x800 sparse matrix of type '<class 'numpy.float64'>'
	with 2129 stored elements in Compressed Sparse Row format>

In [24]:
reload(naive_bayes)
time_stamp = naive_bayes.save(model, parameters["train_naivebayes"])

<module 'posixpath' from '/home/viky/anaconda3/lib/python3.7/posixpath.py'>


In [25]:
reload(metrics)

log_results, cm_train, cm_test = metrics.log_model_outputs(parameters["train_naivebayes"], metrics_test, metrics_train, time_stamp)

In [26]:
from nodes import reports
reload(reports)

cm_test = cm_test.reset_index()

accuracy_pop_data = reports.get_accuracy_pop_data(cm_test, group_tail_labels=False)

reports.plot_accuracy_pop_chart(accuracy_pop_data, name_suffix="naivebayes_all_classes_including_unknown")

reports.plot_accuracy_pop_noncum_chart(accuracy_pop_data, name_suffix="non_cum_naivebayes_all_classes_including_unknown")

FileNotFoundError: [Errno 2] No such file or directory: 'data/08_reporting/accuracy_pop_chart_cum_chart_naivebayes_all_classes_including_unknown.png'

In [ ]:
accuracy_pop_data

In [ ]:
cm_test